In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import WalmartAmazonStructuredBenchmark

benchmark = WalmartAmazonStructuredBenchmark(data_dir_path="../data/")
benchmark

17:59:43 INFO:Extracting Walmart-Amazon-Structured...
17:59:43 INFO:Reading Walmart-Amazon-Structured row_dict...
17:59:43 INFO:Reading Walmart-Amazon-Structured train.csv...
17:59:43 INFO:Reading Walmart-Amazon-Structured valid.csv...
17:59:43 INFO:Reading Walmart-Amazon-Structured test.csv...


<WalmartAmazonStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Walmart-Amazon/walmart_amazon_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'category', 'brand', 'modelno', 'price']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'title_semantic': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'category': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'category_semantic': {
        'source_attr': 'category',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'brand': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'modelno': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

17:59:44 INFO:For attr=title, computing actual max_str_len
17:59:44 INFO:actual_max_str_len=23 must be even to enable NN pooling. Updating to 24
17:59:44 INFO:For attr=title, using actual_max_str_len=24
17:59:44 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:59:48 INFO:For attr=category, computing actual max_str_len
17:59:48 INFO:actual_max_str_len=15 must be even to enable NN pooling. Updating to 16
17:59:48 INFO:For attr=category, using actual_max_str_len=16
17:59:48 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:59:51 INFO:For attr=brand, computing actual max_str_len
17:59:51 INFO:For attr=brand, using actual_max_str_len=46
17:59:51 INFO:For attr=modelno, computing actual max_str_len
17:59:51 INFO:For attr=modelno, using actual_max_str_len=48
17:59:51 INFO:For attr=price, computing actual max_str_len
17:59:51 INFO:For attr=price, using actual_max_str_len=8


{'title': NumericalizeInfo(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=24, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'title_semantic': NumericalizeInfo(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=10,
    eval_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name)
)

17:59:51 INFO:GPU available: True, used: True
17:59:51 INFO:TPU available: None, using: 0 TPU cores
17:59:51 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

17:59:51 INFO:Train pair count: 576
17:59:51 INFO:Valid pair count: 193
17:59:51 INFO:Test pair count: 193
17:59:54 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 22.4 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
13.4 M    Trainable params
9.0 M     Non-trainable params
22.4 M    Total params


17:59:55 WARNING:true_pair_set has 38 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.2754887342453003,
 'title_semantic': 0.23845140635967255,
 'category': 0.06775396317243576,
 'category_semantic': 0.047843966633081436,
 'brand': 0.1762426495552063,
 'modelno': 0.13707798719406128,
 'price': 0.057141274213790894}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.39671120246659813,
 'valid_f1_at_0.5': 0.7598425196850394,
 'valid_f1_at_0.7': 0.9095607235142119,
 'valid_f1_at_0.9': 0.6120996441281138,
 'valid_pair_entity_ratio_at_0.3': 2.058047493403694,
 'valid_pair_entity_ratio_at_0.5': 0.8311345646437994,
 'valid_pair_entity_ratio_at_0.7': 0.5118733509234829,
 'valid_pair_entity_ratio_at_0.9': 0.23218997361477572,
 'valid_precision_at_0.3': 0.24743589743589745,
 'valid_precision_at_0.5': 0.6126984126984127,
 'valid_precision_at_0.7': 0.9072164948453608,
 'valid_precision_at_0.9': 0.9772727272727273,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 1.0,
 'valid_recall_at_0.7': 0.9119170984455959,
 'valid_recall_at_0.9': 0.44559585492227977}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

18:02:41 INFO:Train pair count: 576
18:02:41 INFO:Valid pair count: 193
18:02:41 INFO:Test pair count: 193


[{'test_f1_at_0.3': 0.4371460928652321,
  'test_f1_at_0.5': 0.8278867102396514,
  'test_f1_at_0.7': 0.9246753246753248,
  'test_f1_at_0.9': 0.5703703703703703,
  'test_pair_entity_ratio_at_0.3': 1.796875,
  'test_pair_entity_ratio_at_0.5': 0.6927083333333334,
  'test_pair_entity_ratio_at_0.7': 0.5,
  'test_pair_entity_ratio_at_0.9': 0.20052083333333334,
  'test_precision_at_0.3': 0.2797101449275362,
  'test_precision_at_0.5': 0.7142857142857143,
  'test_precision_at_0.7': 0.9270833333333334,
  'test_precision_at_0.9': 1.0,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 0.9844559585492227,
  'test_recall_at_0.7': 0.9222797927461139,
  'test_recall_at_0.9': 0.39896373056994816}]